# Eksploracja Danych - Prognozowanie Popytu

Ten notebook zawiera analizę eksploracyjną danych (EDA) dla projektu prognozowania popytu na produkty w handlu.

## Cele analizy:
- Załadowanie i wstępne zapoznanie się z danymi
- Analiza rozkładów zmiennych
- Identyfikacja trendów czasowych
- Wykrywanie sezonowości
- Analiza outlierów i brakujących danych

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Konfiguracja wyświetlania
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

## 1. Ładowanie danych

In [ ]:
# Załadowanie danych Online Retail II

## 2. Podstawowe informacje o danych

In [ ]:
# Podstawowe statystyki opisowe

## 3. Analiza szeregów czasowych

In [ ]:
# Analiza trendów czasowych i sezonowości

## 4. Analiza sprzedaży według krajów

In [ ]:
# Analiza geograficzna sprzedaży

## 5. Analiza produktów i kategorii

In [ ]:
# Analiza najpopularniejszych produktów

## 6. Wnioski z analizy eksploracyjnej

### Główne obserwacje:
- [Do uzupełnienia po analizie]

### Implikacje dla modelowania:
- [Do uzupełnienia po analizie]